# Análise de GWAS nas Coortes do LAGC Usando Regenie
Autores: Rafaella Ormond e José Jaime Martinez-Magaña <br>

***Descrição:***<br>
Nesta análise, um GWAS foi realizado utilizando **Regenie** para uma das coortes do LAGC.

**ENTRADA:**  
1) Dados genéticos no formato PLINK pgen/pvar/psam (.pgen, .pvar, .psam)  
2) Arquivos de fenótipo e covariáveis correspondentes à coorte  
3) Covariáveis para ajuste na análise de GWAS  

**SAÍDA:** Resultados de associação do passo 2 do Regenie (arquivos regenie com estatísticas sumárias)


### ***Requisitos:***

**Regenie** é indicado para uso em coortes com tamanho amostral maior que 300 indivíduos.

### Download do Plink
Podemos baixar as versões 1.9 e 2.0 do Plink seguindo as instruções do site oficial.<br>
Para instalar o plink2 [acesse aqui](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar o plink1.9 [acesse aqui](https://www.cog-genomics.org/plink/1.9/)<br>

### Download do Regenie
Para baixar o **Regenie**, acesse: [link aqui](https://github.com/rgcgithub/regenie)

Para mais detalhes, consulte o plano de análise: [acesse aqui](https://docs.google.com/document/d/1RzD5kBlj9rfiomda1G3NfxYDXLdmIUO7VX0cSNj70Kk/edit?usp=sharing)


### Etapas da Análise:
1) Recode pfile para bfile<br>
2) Recode pgen para bgen<br>
3) Regenie passo 1 (ajuste do modelo de regressão genômica ampla)<br>
4) Regenie passo 2 (teste de associação)


### 1) Recodificar pfile para bfile
**Descrição:**<br>
Recodificação de arquivos em formato PLINK2 para o formato bfile, se necessário.<br>
Ajuste os caminhos de entrada e os nomes dos arquivos conforme sua análise.

In [ ]:
## Transformar pgen em bfile para a Etapa 1
# Esta etapa converte o formato .pgen para o formato .bfile para uso na Etapa 1
# Certifique-se de ajustar os nomes e sufixos dos arquivos de acordo com seus arquivos PLINK

plink2 \
    --pfile cohort_name \
    --maf 0.01 --mac 100 --geno 0.1 --hwe 1e-10 --mind 0.1 \
    --write-snplist --write-samples --no-id-header --make-bed --snps-only 'just-acgt' --max-alleles 2 \
    --out cohort_name

### 2) Recodificar pgen para bgen  
**Descrição:**<br>
Esta etapa recodifica arquivos no formato PLINK2 (`.pgen`) para o formato BGEN (`.bgen`) para uso na Etapa 2.  
Ajuste os nomes dos arquivos e caminhos de acordo com o seu conjunto de dados.

**Exemplo de comando:**
```bash
plink2 --pfile cohort_name \
       --export bgen-1.2 'bits=8' \
       --out cohort_name

In [ ]:
## Recodificar pgen para bgen  
# O arquivo pgen apresentou um erro no arquivo psam  
# Certifique-se de ajustar os nomes e sufixos dos arquivos de acordo com seus arquivos PLINK

plink2 --pfile cohort_name --export bgen-1.1 --snps-only 'just-acgt' --max-alleles 2 --out cohort_name --threads 30


# Regenie

Para as etapas 1 e 2, arquivos de fenótipos e covariáveis serão usados, ambos separados por sexo.  

**Arquivos de fenótipos (`inph`)** devem incluir as seguintes colunas:  
`FID`, `IID`, `Pheno1`, `Pheno2`  

Devem existir dois arquivos separados:  
- `inph_bt`: para **traços binários**  
- `inph_qt`: para **traços quantitativos**  

Os fenótipos devem ser selecionados e ajustados com base nos disponíveis na sua coorte e de acordo com o plano de análise:  
[Acesse o plano de análise aqui](https://docs.google.com/document/d/1RzD5kBlj9rfiomda1G3NfxYDXLdmIUO7VX0cSNj70Kk/edit?usp=sharing)  

**Arquivo de covariáveis (`inco`)** deve incluir as seguintes colunas:  
`FID`, `IID`, `age`, e `PC1` a `PC10`  

As 10 PCs (Componentes Principais) devem ser geradas usando **PC-AiR**.  
Certifique-se de que os indivíduos nos arquivos de fenótipos e covariáveis correspondam.

> **Nota:** !!! Atenção, usaremos SNPs do HapMap para a Etapa 1!!!<br>
> Para HapMap selecionamos 250k SNPs aleatoriamente, por favor faça o download deste arquivo no GitHub [LINK aqui](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/02GWAS/00Regenie/w_hm3_hg38_random250K.snplist)

### 3) Regenie Etapa 1

**Descrição:**<br>
**Etapa 1 – Ajuste do modelo de regressão para o genoma completo**<br>
Esta etapa ajusta um modelo de regressão ridge ao longo do genoma usando dados genéticos.  
O objetivo é estimar predições individuais dos fenótipos (leave-one-chromosome-out, LOCO) considerando a estrutura populacional e o grau de parentesco.  
**Entrada:** dados genéticos em formato PLINK e arquivos de fenótipos/covariáveis.  
**Saída:** predições LOCO que serão usadas como covariáveis na Etapa 2.

> **Nota:** !!! Atenção, usaremos SNPs do HapMap para a Etapa 1!!!<br>
> Para HapMap, selecionamos 250k SNPs aleatoriamente. Por favor, faça o download deste arquivo no GitHub [LINK aqui]()

> **Nota:** <br>
> Os arquivos `"inph"` contêm as informações de fenótipos<br>
> Os arquivos `"inco"` contêm as informações de covariáveis, `"bt"` para traços binários e `"qt"` para traços quantitativos<br>
> **A análise precisa ser separada por sexo**


In [ ]:
## Definir parâmetros
# Definir dados genotéticos
# Ajuste de acordo com seus dados
# O "inge" deve ser o prefixo dos arquivos plink
inge="caminho/cohort_name"

# Os arquivos "inph" contêm as informações de fenótipos, "bt" para traços binários e "qt" para traços quantitativos, separados por sexo
# Definir arquivos de fenótipos e covariáveis para mulheres
female_inph_bt="caminho/cohort_female_pheno_bt_forregenie.txt"
female_inph_qt="caminho/cohort_female_pheno_qt_forregenie.txt"
female_inco="caminho/cohort_female_covar_forregenie.txt"

# Definir arquivos de fenótipos e covariáveis para homens
male_inph_bt="caminho/cohort_male_pheno_bt_forregenie.txt"
male_inph_qt="caminho/cohort_male_pheno_qt_forregenie.txt"
male_inco="caminho/cohort_male_covar_forregenie.txt"

# Definir saída para a Etapa 1
out_step1_qt_female="caminho/cohort/out_step1_qt_female"
out_step1_bt_female="caminho/cohort/out_step1_bt_female"
out_step1_qt_male="caminho/cohort/out_step1_qt_male"
out_step1_bt_male="caminho/cohort/out_step1_bt_male"

# Definir saída para a Etapa 2
out_step2_qt_female="caminho/cohort/out_step2_qt_female"
out_step2_bt_female="caminho/cohort/out_step2_bt_female"
out_step2_qt_male="caminho/cohort/out_step2_qt_male"
out_step2_bt_male="caminho/cohort/out_step2_bt_male"

## Adicionar zeros ao FID se ocorrerem erros. Você pode usar o seguinte código em bash
# awk 'BEGIN {OFS="\t"} NR==1 {print "FID", $0; next} {print "0", $0}' "${female_inph_qt}" > tmpfile && mv tmpfile "${female_inph_qt}"

## !!! Aviso, usaremos SNPs do HapMap para a Etapa 1!!!
# Para HapMap, selecionamos 250k SNPs aleatoriamente. Faça o download deste arquivo no GitHub
hapmap="/vast/palmer/scratch/montalvo-ortiz/jjm262/02lagc_smoking_gwas/11references/w_hm3_hg38_random250K.snplist"

## Executando Etapa 1
# Executando para mulheres para traços quantitativos
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${female_inco}\
    --phenoFile ${female_inph_qt}\
    --bsize 400\
    --qt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_qt_female}

# Executando para mulheres para traços binários
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${female_inco}\
    --phenoFile ${female_inph_bt}\
    --bsize 400\
    --iid-only\
    --bt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_bt_female}

# Executando para homens para traços quantitativos
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${male_inco}\
    --phenoFile ${male_inph_qt}\
    --bsize 400\
    --iid-only\
    --qt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_qt_male}

# Executando para homens para traços binários
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${male_inco}\
    --phenoFile ${male_inph_bt}\
    --bsize 400\
    --iid-only\
    --bt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_bt_male}


### 4) Regenie Etapa 2

**Descrição:**  
**Etapa 2 – Teste de associação:** <br>
Esta etapa realiza o GWAS utilizando as predições LOCO da Etapa 1 como offsets em um modelo de regressão linear/firth.  
Cada variante é testada quanto à associação com o traço de interesse.  
Entrada: mesmos arquivos de fenótipos/covariáveis, predições LOCO e dados completos de genótipos.  
Saída: resultados do GWAS (tamanhos de efeito, valores de p, etc.).

> **Nota:** <br>
> Os arquivos `"inph"` contêm as informações de fenótipos<br>
> Os arquivos `"inco"` contêm as informações de covariáveis, `"bt"` para traços binários e `"qt"` para traços quantitativos<br>
> `"bt"` para traços binários e `"qt"` para traços quantitativos<br>
> **A análise precisa ser separada por sexo**


In [ ]:
## Definir parâmetros
# Definir dados de genótipo
# Ajuste de acordo com seus dados
# O "inge" precisa ser o sufixo dos arquivos PLINK
inge="path/cohort_name"

# Os arquivos "inph" contêm as informações de fenótipos, "bt" para traços binários e "qt" para traços quantitativos, separados por sexo
# Definir arquivos de fenótipos e covariáveis para mulheres
female_inph_bt="path/cohort_female_pheno_bt_forregenie.txt"
female_inph_qt="path/cohort_female_pheno_qt_forregenie.txt"
female_inco="path/cohort_female_covar_forregenie.txt"

# Definir arquivos de fenótipos e covariáveis para homens
male_inph_bt="path/cohort_male_pheno_bt_forregenie.txt"
male_inph_qt="path/cohort_male_pheno_qt_forregenie.txt"
male_inco="path/cohort_male_covar_forregenie.txt"

# Definir saída para a etapa 1
out_step1_qt_female="path/cohort/out_step1_qt_female"
out_step1_bt_female="path/cohort/out_step1_bt_female"
out_step1_qt_male="path/cohort/out_step1_qt_male"
out_step1_bt_male="path/cohort/out_step1_bt_male"

# Definir saída para a etapa 2
out_step2_qt_female="path/cohort/out_step2_qt_female"
out_step2_bt_female="path/cohort/out_step2_bt_female"
out_step2_qt_male="path/cohort/out_step2_qt_male"
out_step2_bt_male="path/cohort/out_step2_bt_male"

# Executando para mulheres com traços quantitativos
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${female_inph_qt} \
  --covarFile ${female_inco} \
  --qt \
  --pred ${out_step1_qt_female}_pred.list \
  --bsize 400 \
  --out ${out_step2_qt_female}

# Executando para mulheres com traços binários
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${female_inph_bt} \
  --covarFile ${female_inco} \
  --bt \
  --firth --approx --pThresh 0.01 \
  --pred ${out_step1_bt_female}_pred.list \
  --bsize 400 \
  --out ${out_step2_bt_female}

# Executando para homens com traços quantitativos
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${male_inph_qt} \
  --covarFile ${male_inco} \
  --qt \
  --pred ${out_step1_qt_male}_pred.list \
  --bsize 400 \
  --out ${out_step2_qt_male}

# Executando para homens com traços binários
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${male_inph_bt} \
  --covarFile ${male_inco} \
  --bt \
  --firth --approx --pThresh 0.01 \
  --pred ${out_step1_bt_male}_pred.list \
  --bsize 400 \
  --out ${out_step2_bt_male}
